# Tema 2.2: Atributos, Métodos y Encapsulamiento - Ejercicios Resueltos

### Ejercicio 1

Define una clase `CuentaBancaria` que represente una cuenta de banco.
- Debe tener un atributo público para la moneda (cuyo valor por defecto sea "EUR").
- Un atributo protegido para el titular.
- Un atributo privado para el saldo.

Implementa el constructor obligando a pasar el titular y el saldo inicial. Si al construir el objeto se proporciona un saldo inicial negativo, inicialízalo a 0. Además, crea métodos básicos (getters y setters) para acceder y modificar el saldo. Al intentar asignar un nuevo saldo, no debe permitir valores negativos; si esto ocurre, debe lanzar una excepción `ValueError`.

Crea una instancia de la clase, muestra su saldo, intenta asignarle un saldo negativo (capturando la excepción e imprimiéndola), y por último asígnale un nuevo saldo positivo válido.

In [1]:
class CuentaBancaria:
    def __init__(self, titular, saldo_inicial):
        self.moneda = "EUR"
        self._titular = titular
        if saldo_inicial < 0:
            self.__saldo = 0
        else:
            self.__saldo = saldo_inicial
            
    def get_saldo(self):
        return self.__saldo
        
    def set_saldo(self, nuevo_saldo):
        if nuevo_saldo < 0:
            raise ValueError("El saldo no puede ser negativo.")
        self.__saldo = nuevo_saldo

# Prueba
cuenta = CuentaBancaria("Ana Gómez", 1000)
print(f"Saldo inicial: {cuenta.get_saldo()} {cuenta.moneda}")

try:
    cuenta.set_saldo(-500)
except ValueError as e:
    print(f"Error detectado: {e}")

cuenta.set_saldo(1200)
print(f"Saldo modificado: {cuenta.get_saldo()} {cuenta.moneda}")

Saldo inicial: 1000 EUR
Error detectado: El saldo no puede ser negativo.
Saldo modificado: 1200 EUR


### Ejercicio 2

Modifica la clase anterior (`CuentaBancaria`) para que, en lugar de usar métodos tradicionales (`get_saldo()`, `set_saldo()`), utilice propiedades (el decorador `@property` y su *setter* asociado) para gestionar el acceso y modificación del saldo.

La validación de no aceptar saldos negativos en el *setter* debe mantenerse lanzando un `ValueError`. Comprueba que todo funciona correctamente en el bloque principal accediendo al saldo y modificándolo como si fuera un atributo público más, y capturando la excepción al forzar un saldo negativo.

In [2]:
class CuentaBancaria:
    def __init__(self, titular, saldo_inicial):
        self.moneda = "EUR"
        self._titular = titular
        self.__saldo = saldo_inicial if saldo_inicial >= 0 else 0
            
    @property
    def saldo(self):
        return self.__saldo
        
    @saldo.setter
    def saldo(self, nuevo_saldo):
        if nuevo_saldo < 0:
            raise ValueError("El saldo no puede ser negativo.")
        self.__saldo = nuevo_saldo

# Prueba
cuenta = CuentaBancaria("Luis Pérez", 1500)
print(f"Saldo inicial: {cuenta.saldo} {cuenta.moneda}")  # Uso como property

try:
    cuenta.saldo = -200    # Uso como property provocando error
except ValueError as e:
    print(f"Error detectado: {e}")
    
cuenta.saldo = 2000    # Uso como property válido
print(f"Saldo final: {cuenta.saldo} {cuenta.moneda}")

Saldo inicial: 1500 EUR
Error detectado: El saldo no puede ser negativo.
Saldo final: 2000 EUR


### Ejercicio 3

Crea una clase `SistemaAlarma` que combine atributos y métodos con diferentes niveles de visibilidad:
- Deberá tener un atributo público para la ubicación.
- Un atributo protegido para el estado (`activa`, que es un booleano inicialmente `False`).
- Un atributo privado para el PIN (una cadena de 4 dígitos suministrada al inicializar).
- Añade un método privado que reciba un PIN y devuelva `True` si coincide con el PIN privado, o `False` en caso contrario.
- Añade métodos públicos para activar y desactivar la alarma. Ambos métodos recibirán un PIN y utilizarán el método privado para comprobarlo. Si es correcto, modificarán el estado protegido de la alarma según corresponda; si no lo es, lanzarán un `ValueError`.

Prueba a crear una alarma e intenta modificar su estado tanto con un PIN incorrecto (capturando el error e imprimiendo la excepción) como con el PIN correcto.

In [3]:
class SistemaAlarma:
    def __init__(self, ubicacion, pin):
        self.ubicacion = ubicacion
        self._activa = False
        self.__pin = pin
        
    def __verificar_pin(self, pin_intento):
        return self.__pin == pin_intento
        
    def activar_alarma(self, pin_intento):
        if self.__verificar_pin(pin_intento):
            self._activa = True
        else:
            raise ValueError("PIN incorrecto. No se pudo activar la alarma.")
            
    def desactivar_alarma(self, pin_intento):
        if self.__verificar_pin(pin_intento):
            self._activa = False
        else:
            raise ValueError("PIN incorrecto. No se pudo desactivar la alarma.")

# Prueba
alarma = SistemaAlarma("Puerta Principal", "1234")

try:
    alarma.activar_alarma("0000")  # Fallará
except ValueError as e:
    print(f"Error detectado: {e}")

alarma.activar_alarma("1234")  # Funcionará
print(f"¿Alarma en '{alarma.ubicacion}' activa? {alarma._activa}")

alarma.desactivar_alarma("1234") # Funcionará
print(f"¿Alarma en '{alarma.ubicacion}' activa? {alarma._activa}")

Error detectado: PIN incorrecto. No se pudo activar la alarma.
¿Alarma en 'Puerta Principal' activa? True
¿Alarma en 'Puerta Principal' activa? False


### Ejercicio 4

Define una clase `ConexionRed` para gestionar direcciones de servidores.
- Debe llevar un control de cuántas conexiones activas hay en total usando un atributo estático privado (inicializado a 0).
- Cada vez que se instancie una `ConexionRed` se guardará una dirección IP (atributo de instancia) y el contador estático debe aumentar en 1.
- Crea un método de instancia para cerrar la conexión que solo disminuya el contador en 1.
- Crea un método estático que devuelva el valor actual del contador de conexiones activas.

Pruébalo creando al menos dos conexiones, cerrando alguna y consultando el total con el método estático en varios puntos del proceso (puedes utilizar `print` únicamente en este bloque principal para visualizar los resultados).

In [4]:
class ConexionRed:
    __conexiones_activas = 0
    
    def __init__(self, direccion_ip):
        self.direccion_ip = direccion_ip
        ConexionRed.__conexiones_activas += 1
        
    def cerrar_conexion(self):
        if ConexionRed.__conexiones_activas > 0:
            ConexionRed.__conexiones_activas -= 1
            
    @staticmethod
    def obtener_conexiones_activas():
        return ConexionRed.__conexiones_activas

# Prueba
print("Activas inicialmente:", ConexionRed.obtener_conexiones_activas())
c1 = ConexionRed("192.168.1.10")
print(f"Conexión 1 instanciada ({c1.direccion_ip})")
c2 = ConexionRed("192.168.1.20")
print(f"Conexión 2 instanciada ({c2.direccion_ip})")

print("Total activas tras crear dos:", ConexionRed.obtener_conexiones_activas())

c1.cerrar_conexion()
print("La conexión 1 se cerró.")
print("Total activas tras cerrar una:", ConexionRed.obtener_conexiones_activas())

Activas inicialmente: 0
Conexión 1 instanciada (192.168.1.10)
Conexión 2 instanciada (192.168.1.20)
Total activas tras crear dos: 2
La conexión 1 se cerró.
Total activas tras cerrar una: 1


### Ejercicio 5

Diseña una clase `Empleado` que combine varios tipos de métodos:
- Atributos de instancia privados para nombre y salario.
- Atributo estático definiendo un `salario_minimo` (ej: 1200).
- Método de instancia que devuelva (en un formato legible, sin usar prints) la información del empleado.
- Método de clase (factoría) que reciba una cadena con el formato `"Nombre,Salario"` (ej: `"Ana,1500"`) y devuelva una instancia de la clase parseando esos datos correspondientes.
- Método estático que devuelva `True` si el salario proporcionado es mayor o igual al salario mínimo predefinido, y `False` de lo contrario.

En el código principal, valida numéricos sueltos usando el método estático, crea un empleado usando la inicialización clásica, otro usando el método factoría, y finalmente muestra por pantalla la información de ambos.

In [5]:
class Empleado:
    salario_minimo = 1200
    
    def __init__(self, nombre, salario):
        self.__nombre = nombre
        self.__salario = salario
        
    def obtener_info(self):
        return f"Empleado: {self.__nombre} - Salario: {self.__salario}€"
        
    @classmethod
    def desde_cadena(cls, cadena):
        partes = cadena.split(",")
        nombre = partes[0]
        salario = float(partes[1])
        return cls(nombre, salario)
        
    @staticmethod
    def es_salario_valido(salario):
        return salario >= Empleado.salario_minimo

# Prueba
print("¿Es válido un salario de 1500?:", Empleado.es_salario_valido(1500))
print("¿Es válido un salario de 1000?:", Empleado.es_salario_valido(1000))

e1 = Empleado("Carlos", 1400)
e2 = Empleado.desde_cadena("Laura,1800")

print(e1.obtener_info())
print(e2.obtener_info())

¿Es válido un salario de 1500?: True
¿Es válido un salario de 1000?: False
Empleado: Carlos - Salario: 1400€
Empleado: Laura - Salario: 1800.0€


### Ejercicio 6

Diseña un sistema para simular un Entorno de Hogar Inteligente con dos clases (no hay requisito de herencia entre ellas): `Hub` y `Dispositivo`. Enfócate en el **diseño orientado a objetos** considerando qué niveles de visibilidad tienen sentido, cómo proteger atributos privados con propiedades, y qué tipo de métodos usar para cada requerimiento.

1. Oculta el estado de un dispositivo (encendido/apagado) y su identificador interno del exterior e inicializa el dispositivo con estado apagado.
2. Proporciona una forma segura (mediante una propiedad) de consultar el estado de encendido sin posibilidad de que se pueda sobreescribir este estado con una asignación igual (ej: `d.estado = ...`), y proporciona métodos públicos para realizar explícitamente las acciones de encender y apagar.
3. Los dispositivos normalmente se crean con un identificador interno y su nombre; ofrece también un constructor alternativo rápido (factoría) que asigne el nombre y guarde un identificador `"Desconocido"` por defecto.
4. El `Hub` se encarga de gestionar localmente una lista de dispositivos transitoriamente. Implementa un método en esta clase para ir añadiendo nuevos dispositivos a su control.
5. El `Hub` debe ofrecer un método que devuelva el número de dispositivos que tiene a su cargo.
6. El `Hub` deberá llevar un control general (y accesible en cualquier momento de forma global) de cuántos *hubs* se han fabricado en todo el sistema.
7. Además, un `Hub` ofrecerá el método público general para "apagar el sistema", el cual debe recorrer todos sus dispositivos vinculados e interrumpir su estado usando las herramientas públicas diseñadas.

Escribe el código de estas clases y un pequeño programa principal donde crees un hub, instancies dispositivos de distintas formas, los vincules al hub, y manipules su encendido y apagado para visualizar su resultado final. Comprueba también que el número de dispositivos del hub se calcula correctamente.

In [6]:
class Dispositivo:
    def __init__(self, id_interno, nombre):
        self.__id = id_interno
        self.nombre = nombre
        self.__encendido = False
        
    @classmethod
    def con_id_desconocido(cls, nombre):
        # Constructor / factoría
        return cls("Desconocido", nombre)
        
    @property
    def encendido(self): # Propiedad solo de lectura
        return self.__encendido
        
    def encender(self):
        if not self.__encendido:
            self.__encendido = True
            
    def apagar(self):
        if self.__encendido:
            self.__encendido = False

class Hub:
    total_hubs = 0 # Atributo estático
    
    def __init__(self, nombre_hub):
        self.nombre_hub = nombre_hub
        self._dispositivos = []
        Hub.total_hubs += 1
        
    def agregar_dispositivo(self, dispositivo):
        self._dispositivos.append(dispositivo)
        
    def numero_dispositivos(self):
        return len(self._dispositivos)
        
    def apagar_todo(self):
        for dispositivo in self._dispositivos:
            dispositivo.apagar()

# Programa demostrativo principal
print(f"Hubs instalados inicialmente: {Hub.total_hubs}")
mi_hub = Hub("Centralita Salón")

# Creación de dispositivo normal y con factoría
d1 = Dispositivo("DEV-001", "Luz Techo Principal")
d2 = Dispositivo.con_id_desconocido("Termostato Inteligente")

mi_hub.agregar_dispositivo(d1)
mi_hub.agregar_dispositivo(d2)

print(f"El hub '{mi_hub.nombre_hub}' tiene {mi_hub.numero_dispositivos()} dispositivos vinculados.")

# Encendemos individualmente 
d1.encender()
d2.encender()

print("Se han encendido varios dispositivos.")
print(f" - {d1.nombre} [{d1.encendido}]")
print(f" - {d2.nombre} [{d2.encendido}]")

# Usamos la funcionalidad colectiva del hub
print("\nApagando todo a través del hub...")
mi_hub.apagar_todo()

print(f" - {d1.nombre} [{d1.encendido}]")
print(f" - {d2.nombre} [{d2.encendido}]")

print(f"\nHubs instalados ahora: {Hub.total_hubs}")

Hubs instalados inicialmente: 0
El hub 'Centralita Salón' tiene 2 dispositivos vinculados.
Se han encendido varios dispositivos.
 - Luz Techo Principal [True]
 - Termostato Inteligente [True]

Apagando todo a través del hub...
 - Luz Techo Principal [False]
 - Termostato Inteligente [False]

Hubs instalados ahora: 1
